# Damage Calculation

This notebook shows a general calculation stream for a nominal damage calculation.

1. Define load cycles
2. Define the material parameter
3. Select the damage calculation method (Miner elementary, Miner-Haibach, ...)
4. Calculate the damage for every load level and the damage sum


In [ ]:
import numpy as np
import pandas as pd
import pylife.strength.fatigue
import pylife.vmap
import pylife.stress.equistress
import plotly.express as px
import pylife.mesh.meshsignal
import pylife.mesh.hotspot
import pylife.vmap
import pyvista as pv

### 1. Define load cycles


In [ ]:
load_cycles = pd.DataFrame(
    {
        "range": [100, 50, 75, 25],
        "mean": 0.0,
        "cycles": [1e3, 5e3, 10e3, 25e3],
    },
    index=pd.Index([1, 2, 3, 4], name="load_event")
)

In [ ]:
load_cycles

In [ ]:
vm_mesh = pylife.vmap.VMAPImport("two_notched_beam.vmap")
mesh = (vm_mesh.make_mesh('beam', 'STATE-1')
        .join_coordinates()
        .join_variable('STRESS_CAUCHY')
        .to_frame())

In [ ]:
mesh['mises'] = mesh.equistress.mises() / 1000

In [ ]:
mesh

In [ ]:
grid = pv.UnstructuredGrid(*mesh.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=mesh.groupby('element_id')['mises'].mean().to_numpy(), show_edges=True)
plotter.add_scalar_bar()
plotter.show()

In [ ]:
scaled_collective = load_cycles.load_collective.scale(mesh['mises'])
# scaled_collective.amplitude

## 2. Define the material parameter


In [ ]:
k_1 = 8
mat = pd.Series(
    {
        "k_1": k_1,
        'k_2' : 20,
        "ND": 1.0e6,
        "SD": 400.0,
        "TN": 12.0,
        "TS": 1.1,
    }
)
# display(mat)

In [ ]:
damage = mat.fatigue.miner_original().damage(scaled_collective)
# damage

In [ ]:
grouped_damage = damage.groupby(["load_event", "element_id"]).mean().groupby("element_id").sum()
grouped_damage[grouped_damage > 0.0]

In [ ]:
mesh_result = mesh.join(grouped_damage)
grid = pv.UnstructuredGrid(*mesh_result.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=grouped_damage.to_numpy(), show_edges=True)
plotter.add_scalar_bar()
plotter.show()